# Writing extensions

An extension might be desireable when, for example, signal setters are to be overriden with guard routines, or a latency parameter is to be maintained along for pipelined signal chains.

Like MyHDL, a lot of implicit configuration is buried in a `Signal` type. Therefore, special properties can be attached to derivations of the `Signal` class without actually changing the hardware description.

## Signal extensions

Typically, a class factory routine as the following is sufficient, to create an augmented class of a base class.

First, we create a MixIn style class with no direct derival:

In [1]:
class DebugMixin:
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.usage_count = 0
        self.super = super()

    @classmethod
    def _set(cls, this, val):
        return cls._sig.set(this, val)

    def set(self, val):
        ret = self._set(self, val)
        print("DEBUG SET %s = %s" % (self.identifier, val))
        self.usage_count += 1
        return ret

Then we create a type wrapper that creates a new class type from a specified Signal type:

In [2]:
def debug_setter(sig):
    bases = (DebugMixin, sig)
    cls = type("DebugSetter" + sig.__name__, bases, {})
    cls._sig = sig # Store original signal type
    return cls

In [3]:
from cyhdl import Signal, ClkSignal, intbv

DS = debug_setter(Signal)
DC = debug_setter(ClkSignal)
s = DS(intbv()[4:])
c = DC()

Now we can show the behaviour of this signal in IRL (try repeated execution of the cell below):

In [4]:
action_s = s.set(15)
action_s.evaluate()
s.usage_count

DEBUG SET s_c209 = 15


1

### Example from the library

A questionable example for HDL minded assignment style is found in the library below. This allows assignments using the `<=` operator.

**This is not a recommended code style and can lead to issues**

In [5]:
from myirl.library.style_hdl import _hdlstyle
from myirl.kernel.sig import SigAssign

HS = _hdlstyle(Signal)

s = HS(intbv()[8:], name = 's')
t = HS(intbv()[12:].signed(), name = 't_signed')

actions = [
    s <= 128, t <= s.signed()
]

for a in actions:
    v = a.evaluate()
    print(a, '=>', int(v))

assert v == -128

[ Maybe s <= C:128 ] => 128
[ Maybe t_signed <= SGN(<s>) ] => -128


## Generator extensions

Some constructs might be better represented by a pretty python'ish data type instead of an elaborate hardware description.

Thus, logic extensions can easily be constructed by derivation from a `sensitivity.Generator`.

Note that a generator construct is a priori **not transpiled**, i.e. the decorated function typically returns an IRL construct.

If a custom generator requires CyHDL syntax inside its decorated function, it must be registered in particular. This is not covered here.

In [6]:
from myirl.kernel.sensitivity import Generator, genprocess_ctx

class MyGenerator(Generator):
    def __init__(self, func, idata, odata):
        self.func = func
        self.idata = idata
        self.odata = odata
        # Important to init with func, not self!
        # Otherwise we'll run into recursion.
        super().__init__(func)
        
    def __call__(self, context):
        d = self.func(context)
        _in, _out = self.idata, self.odata
        
        expr = _out.set(_in ^ d)
        
        @genprocess_ctx(_in)
        def user_proc(context):
             yield [ expr ]

        self.add_instance(user_proc)


### Details

When a generator class is instanced inside a hardware unit, it is not generating any logic **until** it is casted by the parenting hardware generation context. This context is in general the top design module, however it can also be the immediate parenting hardware generator for custom type generators. The decision which context is chosen is made by the parenting generator.

A Generator using a function is normally accompanied by a decorator construct for generic usage as follows:

In [7]:
def mygen(i, o):
    def _proc_deco(func):
        return MyGenerator(func, i, o)
    return _proc_deco

Finally, we create an example block making use of the generator:

In [8]:
from cyhdl import *

@block
def unit_ext():
    a, b = [ Signal(intbv()[5:]) for _ in range(2) ]
    
    @mygen(a, b)
    def worker(context):
        if context.target.lang == 'VHDL':
            print("Allow VHDL constructs")
        elif context.target.lang == 'Verilog':
            print("Allow Verilog specific constructs")
        return a | b
    
    return instances()
        

/home/cyrite/.local/lib/python3.10/site-packages/cyritehdl-0.1b0-py3.10-linux-x86_64.egg/myirl/emulation/myhdl2irl.py:582: UserWarning: Not translating unregistered decorator function`mygen`
  warnings.warn("Not translating unregistered decorator function`%s`" % n)


Elaborating this construct into Verilog:

In [9]:
uut = unit_ext()
files = uut.elab(targets.Verilog)

Allow Verilog specific constructs
 Writing 'unit_ext' to file /tmp/myirl_unit_ext_iz44ne1y/unit_ext.v 


The resulting Verilog output:

In [10]:
!cat {files[0]}

// File generated from source:
//     /tmp/ipykernel_814/224027657.py
// (c) 2016-2022 section5.ch
// Modifications may be lost, edit the source file instead.

`timescale 1 ns / 1 ps
`include "aux.v"
// Architecture cyriteHDL

module unit_ext
    ();
    // Local type declarations
    // Signal declarations
    wire [4:0] b;
    wire [4:0] a;
    
    always @ (a) begin : WORKER_USER_PROC
        b = (a ^ (a | b)); /*Generatorprocess 'worker_user_proc'*/
    end
endmodule // unit_ext


## Further examples

As this is not of real world use, more examples are found in the `myirl.library`:

* [LUT generators](../src/myirl/library/lut.py)
* [Hardware sequence generators](../src/myirl/library/custom_generators.py)

For usage, please refer to the corresponding test units.

They are normally found in the `myirl/library/test` directory, however, migration may not be complete, see also `myirl/test`.